In [1]:
timestamp = "2023-03-21 22:00" # Enter timestamp in UTC in this format
api_key = "your API key" # Enter your API key 

In [ ]:
# format for api_key
# api_key = "xxxxxxxx-yyyy-zzzz-aaaa-bbbbbbbbbbbb"

In [2]:
pip install shroomdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from shroomdk import ShroomDK

# Initialize `ShroomDK` with your API Key
sdk = ShroomDK(api_key)
my_address = "0xFeffc0E5C9575576C1922978102afa2D803Dc93F" # garden point contract

sql_blocknumber = f"""
    SELECT
        block_timestamp, 
        block_number
    FROM ethereum.core.ez_nft_transfers
    WHERE date_trunc('minute', block_timestamp) = '{timestamp}'
    ORDER BY block_timestamp DESC
    LIMIT 1
"""

query_result_timestamp = sdk.query(sql_blocknumber)

for record in query_result_timestamp.records:
    snapshot = record['block_number']

In [4]:
# Parameters can be passed into SQL statements 
# via native string interpolation

sql = f"""
    WITH wallets as(
    SELECT 
        nft_to_address as wallet, 
        max(block_number) as last_transfer 
    FROM ethereum.core.ez_nft_transfers 
    WHERE nft_address = LOWER('{my_address}')
    GROUP BY wallet
    ),

    first_data as(
    SELECT
        nft_to_address as owner,
        tokenid
    FROM ethereum.core.ez_nft_transfers
    WHERE nft_address = LOWER('{my_address}')
    AND block_number <= '{snapshot}'
    QUALIFY RANK() OVER (
        PARTITION BY tokenid 
        ORDER BY block_number DESC, event_index DESC) = 1        
    )
    SELECT *
    FROM first_data
    ORDER BY owner
    
"""

# Run the query against Flipside's query engine 
# and await the results
query_result_set = sdk.query(sql)

In [5]:
import pandas as pd

# create empty data frame container
data = []

# convert sql run result into pandas dataframe
for record in query_result_set.records:
    owner = record['owner']
    tokenid = record['tokenid']
    data.append({'owner': owner, 'token_id': tokenid})
df = pd.DataFrame(data)

# convert token id data type from object/string to integer
df['token_id'] = df['token_id'].astype(int)

In [6]:
# import garden point token metadata from github repository

token_metadata_url = 'https://raw.githubusercontent.com/fcitra/GP-snapshot/main/GardenPoint%20Covers.csv'
token_metadata_df = pd.read_csv(token_metadata_url)

In [7]:
# merge token holders' snapshot with token metadata
merged_df = pd.merge(df, token_metadata_df, on ='token_id', how = 'left')

# group token quantity based on holder wallet and rarity type
grouped_df = merged_df.groupby(['owner', 'Metadata'])['Metadata'].size()

# unstack grouped rows into columns
df_unstack = grouped_df.unstack()

# change NaN value into 0 
df_unstack = df_unstack.fillna(0)

In [8]:
# export dataframe to csv file 

df_unstack.to_csv('snapshot_rarity.csv', index = True)